In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)
#print(school_data.head())
#print(student_data.head())
# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

### District Summary
columns = ['Total Students']
# Create a high level snapshot (in table form) of the district's key metrics, including:
# Total Schools
Total_Schools=school_data_complete[['school_name']].nunique()
Total_Schools=Total_Schools.iloc[0].item()
#Total_Schools


# Total Students
Total_Students=school_data_complete[['Student ID']].nunique()
Total_Students=Total_Students.iloc[0].item()
#Total_Students

# Total Budget
Total_Budget=school_data[['budget']].sum()
Total_Budget=Total_Budget.iloc[0].item()
#Total_Budget

# Average Math Score
Avg_Math_Score=school_data_complete[['math_score']].mean()
Avg_Math_Score=Avg_Math_Score.iloc[0].item()
#Avg_Math_Score

# Average Reading Score
Avg_Read_Score=school_data_complete[['reading_score']].mean()
Avg_Read_Score=Avg_Read_Score.iloc[0].item()
#Avg_Read_Score

# % Passing Math
Pass_Math_Score=school_data_complete.loc[school_data_complete['math_score']>=70]
N_Math_Score=Pass_Math_Score[['math_score']].count()
N_Math_Score=N_Math_Score.iloc[0].item()
Per_Pass_Math=N_Math_Score/Total_Students*100
#Per_Pass_Math

# % Passing Reading
Pass_Read_Score=school_data_complete.loc[school_data_complete['reading_score']>=70]
N_Read_Score=Pass_Read_Score[['math_score']].count()
N_Read_Score=N_Read_Score.iloc[0].item()
Per_Read_Math=N_Read_Score/Total_Students*100
#Per_Read_Math

# Overall Passing Rate (Average of the above two)
Overall=(Per_Pass_Math+Per_Read_Math)/2
#Overall

summary_table = pd.DataFrame({
                                "Total Schools": Total_Schools,
                              "Total Students": [Total_Students],
                              "Total Budget": [Total_Budget],
                              "Average Math Score": [Avg_Math_Score],
                              "Average Reading Score": [Avg_Read_Score],
                              "% Passing Math": Per_Pass_Math,
                              "% Passing Reading": [Per_Read_Math],
                              "% Overall Passing Rate": [Overall]
                            })
summary_table

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,80.393158


In [2]:
### School Summary

# Create an overview table that summarizes key metrics about each school, including:
#   * School Name
#   * School Type
#   * Total Students
Total_Stu=school_data_complete.groupby(['school_name'])['Student ID'].count()
Total_Stu=pd.DataFrame(Total_Stu).rename(columns={'Student ID':'Total Students'})
Total_Stu

#   * Total School Budget
school_demo=school_data[['school_name','type','budget']]
school_demo=pd.DataFrame(school_demo).set_index('school_name').rename(columns={'type':'School Type','budget':'Total School Budget'})
school_demo

#   * Average Math Score
Avg_Math_BySchool=school_data_complete.groupby(['school_name'])['math_score'].mean()
Avg_Math_BySchool=pd.DataFrame(Avg_Math_BySchool).rename(columns={'math_score':'Average Math Score'})
Avg_Math_BySchool

#   * Average Reading Score
Avg_Read_BySchool=school_data_complete.groupby(['school_name'])['reading_score'].mean()
Avg_Read_BySchool=pd.DataFrame(Avg_Read_BySchool).rename(columns={'reading_score':'Average Reading Score'})
Avg_Read_BySchool

#   * % Passing Math
Pass_Math_BySchool=school_data_complete.loc[school_data_complete['math_score']>=70]
N_Pass_Math_BySchool=Pass_Math_Score.groupby(['school_name'])['math_score'].count()
N_Pass_Math_BySchool=pd.DataFrame(N_Pass_Math_BySchool).rename(columns={'math_score':'count passing math'})
N_Pass_Math_BySchool


#   * % Passing Reading
Pass_Read_BySchool=school_data_complete.loc[school_data_complete['reading_score']>=70]
N_Pass_Read_BySchool=Pass_Read_BySchool.groupby(['school_name'])['reading_score'].count()
N_Pass_Read_BySchool=pd.DataFrame(N_Pass_Read_BySchool).rename(columns={'reading_score':'count passing reading'})
N_Pass_Read_BySchool

#   * Per Student Budget
#   * Overall Passing Rate (Average of the above two)

merge1 = pd.merge(school_demo, Total_Stu, on="school_name", how="left")
merge1
merge2=pd.merge(merge1, Avg_Math_BySchool, on="school_name", how="left")
merge2
merge3=pd.merge(merge2, Avg_Read_BySchool, on="school_name", how="left")
merge3
merge4=pd.merge(merge3, N_Pass_Math_BySchool, on="school_name", how="left")
merge4
merge5=pd.merge(merge4, N_Pass_Read_BySchool, on="school_name", how="left")
merge5['Per Student Budget']=merge5['Total School Budget']/merge5['Total Students']
merge5['% Passing Math']=merge5['count passing math']/merge5['Total Students']*100
merge5['% Passing Reading']=merge5['count passing reading']/merge5['Total Students']*100
merge5['% Overall Passing Rate']=(merge5['% Passing Math']+merge5['% Passing Reading'])/2
merge5=merge5[['School Type','Total Students','Total School Budget'
               ,'Per Student Budget','Average Math Score','Average Reading Score'
               ,'% Passing Math','% Passing Reading','% Overall Passing Rate']]
School_Summary=merge5
School_Summary['Total School Budget']=School_Summary['Total School Budget'].map("${:,.2f}".format)
School_Summary['Per Student Budget']=School_Summary['Per Student Budget'].map("${:.2f}".format)
#merge5.rename(columns={},index={'school_name': ''},inplace=True)



In [3]:
### Top Performing Schools (By Passing Rate)
Top_Performing=School_Summary.sort_values(['% Overall Passing Rate'],ascending=False).head(5)
#Top_Performing['Total School Budget'].astypes('int64')
Top_Performing

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,95.586652
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,95.290520
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,95.270270
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,95.265668
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,95.203679


In [4]:
### Bottom Performing Schools (By Passing Rate)
Bottom_Performing=School_Summary.sort_values(['% Overall Passing Rate'],ascending=True).head(5)
Bottom_Performing

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,73.293323
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,73.363852
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,73.500171
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,73.639992
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,73.804308


In [5]:
### Math Scores by Grade\*\*

# Create a table that lists the average Math Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
Avg_Math_byGradebySchool=school_data_complete.groupby(['school_name','grade'])['math_score'].mean()
#Avg_Math_byGradebySchool=pd.DataFrame(Avg_Math_byGradebySchool)
Avg_Math_byGradebySchool=Avg_Math_byGradebySchool.unstack()
Avg_Math_byGradebySchool=Avg_Math_byGradebySchool[['9th','10th','11th','12th']]
del Avg_Math_byGradebySchool.index.name
del Avg_Math_byGradebySchool.columns.name
Avg_Math_byGradebySchool

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


In [6]:
### Reading Scores by Grade

# Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.
Avg_Read_byGradebySchool=school_data_complete.groupby(['school_name','grade'])['reading_score'].mean()
#Avg_Math_byGradebySchool=pd.DataFrame(Avg_Math_byGradebySchool)
Avg_Read_byGradebySchool=Avg_Read_byGradebySchool.unstack()
Avg_Read_byGradebySchool=Avg_Read_byGradebySchool[['9th','10th','11th','12th']]
del Avg_Read_byGradebySchool.index.name
del Avg_Read_byGradebySchool.columns.name
Avg_Read_byGradebySchool

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


In [7]:
### Scores by School Spending
School_Summary['Per Student Budget'].max()

School_Summary['Per Student Budget'].min()

spending_bins = [0, 550, 600, 650, 700]
group_names = ["<$550", "$550-600", "$600-650", "$650-700"]

School_Summary['Per Student Budget'] = School_Summary['Per Student Budget'].str.replace(',', '').str.replace('$', '')
Spend_per_student=School_Summary


Spend_per_student = Spend_per_student.convert_objects(convert_numeric=True)
#df.dtypes
Spend_per_student["Spending Ranges (Per Student)"] = pd.cut(Spend_per_student["Per Student Budget"], spending_bins, labels=group_names)
Spend_per_student

#pd.DataFrame(df).dtypes


# Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
#   * Average Math Score
#   * Average Reading Score
#   * % Passing Math
#   * % Passing Reading
#   * Overall Passing Rate (Average of the above two)
Spend_per_student = Spend_per_student.groupby("Spending Ranges (Per Student)", as_index=False)['Average Math Score'
                                                   ,'Average Reading Score'
                                                  ,'% Passing Math'
                                                  ,'% Passing Reading'
                                                  ,'% Overall Passing Rate'].mean()
Spend_per_student.set_index('Spending Ranges (Per Student)')

/Users/dhartipatel804/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  del sys.path[0]


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$550,NaN,NaN,NaN,NaN,NaN
$550-600,83.436210,83.892196,93.541501,96.459627,95.000564
$600-650,79.423466,82.044963,76.832677,86.725974,81.779326
$650-700,76.959583,81.058567,66.218444,81.089710,73.654077


In [8]:
### Scores by School Size

# Repeat the above breakdown, but this time group schools based on a reasonable approximation of school size (Small, Medium, Large).
School_Summary['Total Students'].max()

School_Summary['Total Students'].max()

spending_bins = [0, 1000, 3000, 5000]
group_names = ["small (<1000)", "medium (1000-3000)", "large(3000-5000)"]

School_Size=School_Summary

School_Size = School_Size.convert_objects(convert_numeric=True)
#df.dtypes
School_Size["School Size"] = pd.cut(School_Size["Total Students"], spending_bins, labels=group_names)


School_Size = School_Size.groupby("School Size", as_index=False)['Average Math Score'
                                                   ,'Average Reading Score'
                                                  ,'% Passing Math'
                                                  ,'% Passing Reading'
                                                  ,'% Overall Passing Rate'].mean()
School_Size.set_index('School Size')

/Users/dhartipatel804/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: convert_objects is deprecated.  To re-infer data dtypes for object columns, use DataFrame.infer_objects()
For all other conversions use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  del sys.path[0]


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
medium (1000-3000),81.176821,82.933187,84.649798,91.316412,87.983105
large(3000-5000),77.063340,80.919864,66.464293,81.059691,73.761992


In [9]:
### Scores by School Type

# Repeat the above breakdown, but this time group schools based on school type (Charter vs. District).
School_Summary = School_Summary.groupby("School Type", as_index=False)['Average Math Score'
                                                   ,'Average Reading Score'
                                                  ,'% Passing Math'
                                                  ,'% Passing Reading'
                                                  ,'% Overall Passing Rate'].mean()


School_Summary.set_index('School Type')

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757


In [10]:
# You must include a written description of at least two observable trends based on the data.

# Observation 1: School with a smaller number of students have better average grades in all categories;
#     average math score, average reading score, percent passing math, percent passing reading, and percent overall passing rate.
    
# Observation 2: School that spend more money per student do not have higher average math and reading scores.
#     In fact, schools that spend less per students actually have higher average math and reading scores.
#     This is not to say that spending less money per student increases the average math and reading scores,
#     other facts need to be taken into consideration. For example, the size and budget of the schools together. 

